# Test Schema View
Notebook that tests how to use the linkml shema view utility for interrogating the NMDC Schema.

In [3]:
from linkml_runtime.utils.schemaview import SchemaView

In [4]:
view = SchemaView('../src/schema/nmdc.yaml')

In [5]:
view.imports_closure()

Loading schema external_identifiers from ../src/schema/nmdc.yaml
Loading schema core from ../src/schema/nmdc.yaml
Loading schema prov from ../src/schema/nmdc.yaml
Loading schema basic_slots from ../src/schema/nmdc.yaml
Loading schema https://w3id.org/linkml/types from ../src/schema/nmdc.yaml
Loading schema annotation from ../src/schema/nmdc.yaml
Loading schema workflow_execution_activity from ../src/schema/nmdc.yaml
Loading schema mixs from ../src/schema/nmdc.yaml


['NMDC',
 'external_identifiers',
 'core',
 'prov',
 'basic_slots',
 'linkml:types',
 'annotation',
 'workflow_execution_activity',
 'mixs']

### Get list of classes (limit to 5)

In [38]:
list(view.all_class())[0:5]

['database', 'data object', 'biosample', 'study', 'biosample processing']

### Get list of all slots (limit to 5)

In [39]:
list(view.all_slot())[0:5]

['object set',
 'biosample set',
 'study set',
 'data object set',
 'genome feature set']

### Check if slot is mulitvalued

In [9]:
view.get_slot('study set').multivalued

True

### Determine range of slot

In [11]:
view.get_slot('has input').range

'named thing'

### Determine range as specified in the slot_usage

In [27]:
view.get_class('biosample processing').slot_usage['has input'].range

'biosample'

### If slot is not in slot_usage, an error will be throw. So, use function to determine slot range.

In [48]:
def get_class_slot_range(view, class_name, slot_name):
    if slot_name in view.get_class(class_name).slot_usage:
        return view.get_class(class_name).slot_usage[slot_name].range
    else:
        return view.get_slot(slot_name).range

print(get_class_slot_range(view, 'biosample processing', 'has input'))
print(get_class_slot_range(view, 'biosample processing', 'has output'))
        

biosample
named thing


### Test for non-existent slots

In [50]:
view.get_slot('foo') ## nothing returned

In [52]:
'foo' in view.get_class('biosample processing').slots

False

### Use induced_slot method to find range

In [65]:
 help(SchemaView.induced_slot)

Help on _lru_cache_wrapper in module linkml_runtime.utils.schemaview:

induced_slot(self, slot_name: Union[linkml_runtime.linkml_model.meta.SlotDefinitionName, str], class_name: Union[linkml_runtime.linkml_model.meta.ClassDefinitionName, str] = None, imports=True) -> linkml_runtime.linkml_model.meta.SlotDefinition
    Given a slot, in the context of a particular class, yield a dynamic SlotDefinition that
    has all properties materialized.
    
    This makes use of schema slots, such as attributes, slot_usage. It also uses ancestor relationships
    to infer missing values
    
    :param slot_name: slot to be queries
    :param class_name: class used as context
    :param imports: include imports closure
    :return: dynamic slot constructed by inference



In [66]:
view.induced_slot('has input', 'biosample processing').range ## BUG!

'named thing'